In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json
from pprint import pprint


# Import API key
from config import g_key

In [31]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame

# Create variable & csv path to cities
worldcities_to_load = "../WeatherPy/worldcities.csv"
worldcities = pd.read_csv(worldcities_to_load)
worldcities.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [32]:

# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = worldcities[["Lat", "Lng"]]

humidity = worldcities["Humidity"]

In [34]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)


# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)



# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig
max_intensity



290

In [40]:
### Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

# Define first Dataframe, focusing on Max Temp, Wind Speed, and Cloudines

# Create base DataFrame
worldcities.head()

# Isolate cities with ideal conditions - wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
ideal_city_df = worldcities.loc[(worldcities["Wind Speed"] <= 10) & (worldcities["Cloudiness"] == 0) & \
                                   (worldcities["Max Temp"] >= 70) & (worldcities["Max Temp"] <= 80)].dropna()

ideal_city_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


In [44]:
### Hotel Map
# Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

#Import API key
from config import g_key

# Configure gmaps; addressing issues with g_key
gmaps.configure(confi = g_key)

# Create hotel file
hotel_df = ideal_city_df.reset_index(drop=True)

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

#Define params
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}
    
# Loop and use lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
        # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
hotel_df



TypeError: configure() got an unexpected keyword argument 'weather_api_key'